In [2]:
import pandas as pd
import json
import os
import csv
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [17]:
'''
Because of issues importing flickr 8k dataset directly as a folder.
1. Add flickr 8k folder to your google drive.
2. Add a BASE_DIR path by commenting out the exsiting BASE_DIR 

Or if it works for you to add the dataset folder in google colab, go ahead.
'''

# flickr8k dataset (images and caption.txt)
BASE_DIR = '/content/drive/MyDrive/LTU/Deep Learning/Lab 3/flickr 8k' # nebos path
image_folder = os.path.join(BASE_DIR, 'images')
caption_file = os.path.join(BASE_DIR, 'captions.txt')


# transform images to fit ImageNet
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class Flickr8kDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        """
        Args:
            image_folder (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.image_folder = image_folder
        self.transform = transform
        self.image_filenames = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.image_filenames[idx])
        try:
            image = Image.open(img_name).convert('RGB')
            if self.transform:
                image = self.transform(image)
        except IOError:
          print(f'Error opening image {img_name}, skipping')
          return None

        return image

# Define the transformation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Initialize dataset
flickr_dataset = Flickr8kDataset(image_folder=image_folder, transform=transform)

# Load and organize captions
def load_captions(filename):
    captions_dict = {}
    with open(filename, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            if len(row) < 2:
                continue
            image_id, caption = row[0], row[1]
            if image_id in captions_dict:
                captions_dict[image_id].append(caption)
            else:
                captions_dict[image_id] = [caption]
    return captions_dict

captions = load_captions(caption_file)


38009
Batch Images shape torch.Size([32, 3, 224, 224])
Batch captions shape: 32


In [ ]:
#Encoder, Convolutional Neural Network, Pretrained ImageNet
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.eval()


In [ ]:
#Decoder, Recurrent Neural Network, LSTM with Attention

In [ ]:
#Beam search, to transform the Decoder's output into a score for each word in the vocabulary